In [1]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

##### 1. Load one vector embedding

In [2]:
#Get embeddings and Chroma to load the vector store
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [3]:
embeddings_model = OpenAIEmbeddings()

In [4]:
vectorstore = Chroma(persist_directory='data/chroma/1Causal_Inference',embedding_function=embeddings_model)

##### 2. Setup the OpenAI Chat LLM

In [5]:
from langchain.chat_models import ChatOpenAI

In [6]:
llm = ChatOpenAI(temperature=0)

##### 3. MultiQuery Retriever

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
mq = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

#### 4. Setup Conversational Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")

##### 5A. My Langchan Chains toward the Agent

In [ ]:
from langchain import LLMMathChain

In [ ]:
llm_math_model = LLMMathChain.from_llm(llm)

In [ ]:
llm_math_model.run(question='What is 3 raised to 3?')

##### 5B. Teaching Agent using MQ for search

In [ ]:
from langchain.chains import LLMChain

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

In [ ]:
teacher_system_template = """
You are a specialized teacher chatbot with expertise in causal inference using Python. 
Your primary role is to guide students through interactive lessons, hands-on exercises, and provide thoughtful insights. 
You will help students understand the underlying principles, methodologies, and practical applications of causal inference.

Your responses should be clear, concise, and tailored to the students' level of understanding, encouraging 
them to think critically and engage with the material. 
Encourage questions and provide examples where necessary to illustrate complex ideas.

When explaining concepts, refer to the internal documents by citing page numbers or section headers as provided from the 
vector store. Do not reference any external links or sources outside the provided internal documents. 
Ensure you added page numbers at the end of your response without exception? Citing content is a must.
"""

teacher_human_template = 'As a teacher of this book answer this {question}'


In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([teacher_system_template,teacher_human_template])

In [ ]:
teacher_llm = LLMChain(llm=llm, prompt=chat_prompt_template)

In [ ]:
question = "What are the benefits of OLS regression with regards to casuality analysis?"

In [ ]:
teacher_llm.run(input_documents=mq.get_relevant_documents(question),question=question)

##### 5C. Reasoning Agent using MQ for search

In [ ]:
reasoning_system_template = """
You are an autoregressive language model that has been fine-tuned with instruction-tuning and RLHF. You carefully provide accurate, factual, thoughtful, nuanced answers, and are brilliant at reasoning. If you think there might not be a correct answer, you say so.

Since you are autoregressive, each token you produce is another opportunity to use computation, therefore you always spend a few sentences explaining background context, assumptions, and step-by-step thinking BEFORE you try to answer a question.

Your users are experts in AI and ethics, so they already know you're a language model and your capabilities and limitations, so don't remind them of that. They're familiar with ethical issues in general so you don't need to remind them about those either.

Don't be verbose in your answers, but do provide details and examples where it might help the explanation. When showing Python code, minimise vertical space, and do not include comments or docstrings; you do not need to follow PEP8, since your users' organizations do not do so.
"""

reasoning_human_template = 'Using all of your reasoning skills answer this {question}'

In [ ]:
chat_prompt_template2 = ChatPromptTemplate.from_messages([reasoning_system_template,reasoning_human_template])

In [ ]:
reasnoning_llm = LLMChain(llm=llm, prompt=chat_prompt_template2,memory=memory)

In [ ]:
question = "What is OLS relevant in casuality analysis?"

In [ ]:
reasnoning_llm.run({'input_documents':mq.get_relevant_documents(question),
                    'question':question})

##### 5D. Current Time Tool as a Python Function

In [ ]:
from langchain.agents import tool

In [ ]:
from datetime import datetime
# DOC STRINGS SHOULD BE VERY DESCRIPTIVE
# IT IS WHAT THE LLM READS TO DECIDE TO USE THE TOOL!
@tool
def get_time(text: str) -> str:
    '''Returns the current time. Use this for any questions
    regarding the current time. Input is an empty string and
    the current time is returned in a string format. Only use this function
    for the current time. Other time related questions should use another tool'''
    return str(datetime.now()).split('.')[0]

In [ ]:
get_time('time')

###### 6. Setup Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")

##### 7. Tools

In [ ]:
from langchain.agents import Tool
from langchain.agents import load_tools

In [ ]:
#tools = load_tools(["llm-math"], llm=llm)
tools = [
    Tool(
        name="Specialised teacher chatbot",
        func=teacher_llm.run,#(input_documents=mq.get_relevant_documents(question),question=question)
        description="useful for when you have a specific question around causal analysis, stats, machine learning, etc",
        return_direct=True,
    ),
    Tool(
        name="Specialised reasoning chatbot",
        func=reasnoning_llm.run, #(input_documents=mq.get_relevant_documents(question),question=question),
        description="useful for questions that require reasoning skills",
        return_direct=True,
    ),
    
     ]

In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [ ]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0)
agent = initialize_agent(tools, 
                         chat_model, 
                         agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         input_documents=mq.get_relevant_documents(input),
                         memory=memory)
agent.run(input = "What is the title of the book you are a teacher on?")

In [ ]:
agent.run(input = "explain to me what that book is about ?")

In [ ]:
memory